Vision Transformer 구현하기

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]
import keras
from keras import layers
from keras import ops
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
num_classes = 100
input_shape = (32, 32, 3)
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
learning_rate = 0.001  # 학습률 설정
weight_decay = 0.0001  # 가중치 감소(정규화) 값 설정
batch_size = 256  # 배치 크기
num_epochs = 10  # 학습 반복 횟수 (실제 학습에서는 100을 권장, 10은 테스트 값)
image_size = 72  # 입력 이미지 크기를 이 크기로 리사이징
patch_size = 6  # 입력 이미지에서 추출할 패치 크기
num_patches = (image_size // patch_size) ** 2  # 총 패치 개수
projection_dim = 64  # 패치 임베딩 차원 크기
num_heads = 4  # 멀티헤드 어텐션에서 사용할 헤드 개수
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Transformer 레이어 크기
transformer_layers = 8  # Transformer 블록 개수
mlp_head_units = 
[2048,1024,]

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),  # 데이터 정규화 (평균 및 분산을 이용해 스케일 조정)
        layers.Resizing(image_size, image_size),  # 이미지 크기를 지정된 크기(image_size)로 조정
        layers.RandomFlip("horizontal"),  # 이미지를 좌우로 무작위 반전 (데이터 증강)
        layers.RandomRotation(factor=0.02),  # 이미지를 약간 회전 (최대 2% 범위)
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),  # 랜덤 줌 (높이와 너비 20%까지 확대/축소)
    ],
    name="data_augmentation",  # 데이터 증강 레이어의 이름 설정
)
# 정규화를 위해 훈련 데이터의 평균과 분산을 계산
data_augmentation.layers[0].adapt(x_train)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):  # 패치(Patch) 추출을 위한 커스텀 Keras 레이어
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size  # 패치 크기 저장
    def call(self, images):  # 입력 이미지에서 패치를 추출하는 함수
        input_shape = ops.shape(images)  # 입력 이미지의 크기 가져오기
        batch_size = input_shape[0]  # 배치 크기
        height = input_shape[1]  # 이미지 높이
        width = input_shape[2]  # 이미지 너비
        channels = input_shape[3]  # 채널 수 (RGB의 경우 3)
        num_patches_h = height // self.patch_size  # 높이 방향 패치 개수
        num_patches_w = width // self.patch_size  # 너비 방향 패치 개수
        # 이미지를 작은 패치들로 분할
        patches = keras.ops.image.extract_patches(images, size=self.patch_size)

In [ ]:
# 패치를 (배치 크기, 패치 개수, 패치 벡터 크기) 형태로 변환
        patches = ops.reshape(
            patches,
            (
                batch_size,
                num_patches_h * num_patches_w,  # 전체 패치 개수
                self.patch_size * self.patch_size * channels,  # 패치를 벡터로 변환
            ),
        )
        return patches  # 변환된 패치 반환
    def get_config(self):  # 모델 저장 및 불러오기를 위한 설정 반환
        config = super().get_config()
        config.update({"patch_size": self.patch_size})  # 패치 크기 정보 추가
        return config

In [ ]:
plt.figure(figsize=(4, 4))  # 4x4 크기의 그림(플롯) 생성
image = x_train[np.random.choice(range(x_train.shape[0]))]  # 학습 데이터에서 랜덤한 이미지 선택
plt.imshow(image.astype("uint8"))  # 선택한 이미지를 화면에 표시
plt.axis("off")  # 축 제거 (불필요한 테두리 없앰)
# 선택한 이미지를 지정된 크기로 리사이징
resized_image = ops.image.resize(
    ops.convert_to_tensor([image]), size=(image_size, image_size)
)
# 리사이징된 이미지에서 패치 생성
patches = Patches(patch_size)(resized_image)

In [ ]:
# 패치 관련 정보 출력
print(f"Image size: {image_size} X {image_size}")  # 이미지 크기 출력
print(f"Patch size: {patch_size} X {patch_size}")  # 패치 크기 출력
print(f"Patches per image: {patches.shape[1]}")  # 한 이미지에서 생성된 패치 개수 출력
print(f"Elements per patch: {patches.shape[-1]}")  # 각 패치가 가진 요소 개수 출력

# 패치를 시각적으로 확인
n = int(np.sqrt(patches.shape[1]))  # 패치 개수의 제곱근 계산 (정사각형 형태로 출력하기 위함)
plt.figure(figsize=(4, 4))  # 4x4 크기의 플롯 생성
for i, patch in enumerate(patches[0]):  # 첫 번째 이미지의 패치들을 반복하면서 출력
    ax = plt.subplot(n, n, i + 1)  # n x n 서브플롯 생성
    patch_img = ops.reshape(patch, (patch_size, patch_size, 3))  # 패치를 원래 이미지 형태로 변환
    plt.imshow(ops.convert_to_numpy(patch_img).astype("uint8"))  # 패치를 시각화
    plt.axis("off")  # 축 제거

In [ ]:
class PatchEncoder(layers.Layer):  # 패치 인코딩을 위한 커스텀 Keras 레이어
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches  # 전체 패치 개수 저장
        self.projection = layers.Dense(units=projection_dim)  # 패치를 특정 차원으로 변환하는 완전 연결(Dense) 레이어
        self.position_embedding = layers.Embedding(  # 위치 임베딩을 위한 Embedding 레이어
            input_dim=num_patches, output_dim=projection_dim
        )
    def call(self, patch):
        # 패치의 위치 인덱스 생성 (0부터 num_patches-1까지)
        positions = ops.expand_dims(
            ops.arange(start=0, stop=self.num_patches, step=1), axis=0
        )

In [ ]:
# 각 패치를 projection_dim 차원으로 변환 (선형 변환)
        projected_patches = self.projection(patch)
        
        # 위치 임베딩을 추가하여 패치의 위치 정보를 인코딩
        encoded = projected_patches + self.position_embedding(positions)
        
        return encoded  # 인코딩된 패치 반환
    def get_config(self):  # 모델 저장 및 불러오기를 위한 설정 반환
        config = super().get_config()
        config.update({"num_patches": self.num_patches})  # 패치 개수 정보 추가
        return config

In [ ]:
def create_vit_classifier():  # Vision Transformer(ViT) 분류 모델 생성 함수
    inputs = keras.Input(shape=input_shape)  # 입력층 정의 (이미지 크기: input_shape)
    
    # 데이터 증강 (Data Augmentation) 적용
    augmented = data_augmentation(inputs)
    
    # 이미지를 작은 패치(Patch)로 분할
    patches = Patches(patch_size)(augmented)
    
    # 패치를 Transformer가 처리할 수 있도록 인코딩 (Patch Embedding + Position Encoding)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

In [ ]:
    for _ in range(transformer_layers):
        # Layer Normalization 1
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        # Multi-Head Self-Attention 레이어 생성
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        
        # Skip Connection 1 (잔차 연결)
        x2 = layers.Add()([attention_output, encoded_patches])
        
        # Layer Normalization 2
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP (Feed Forward Network) 추가
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        
        # Skip Connection 2 (잔차 연결)
        encoded_patches = layers.Add()([x3, x2])

In [ ]:
# 최종 특징 벡터 생성 (Batch 크기 × projection_dim 형태의 텐서)

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)  # Flatten하여 1D 벡터로 변환
    representation = layers.Dropout(0.5)(representation)  # 과적합 방지를 위한 Dropout
    
    # MLP 헤드 추가 (Fully Connected Layer)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    
    # 최종 분류 레이어 (num_classes 개의 클래스 출력)
    logits = layers.Dense(num_classes)(features)
    
    # Keras 모델 생성
    model = keras.Model(inputs=inputs, outputs=logits)
    return model  # 모델 반환

In [ ]:
    def run_experiment(model):  # 모델 학습 및 평가를 수행하는 함수
        
    # AdamW 옵티마이저 설정 (Adam + Weight Decay 적용)
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
    
    # 모델 컴파일 (손실 함수 및 평가 지표 설정)
    model.compile(
        optimizer=optimizer,  # AdamW 옵티마이저 사용
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 다중 클래스 분류를 위한 손실 함수
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),  # 정확도(Accuracy)
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),  
	    # Top-5 정확도 (상위 5개 중 정답 포함 여부)
        ],
    )
    
    # 체크포인트(최적 모델 저장) 파일 경로 설정
    checkpoint_filepath = "/content/drive/MyDrive/Colab/checkpoint.weights.h5"

In [ ]:
   # 체크포인트 콜백 설정 (최고 성능 모델 저장)
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,  # 저장 위치
        monitor="val_accuracy",  # 검증 데이터 정확도 기준으로 저장
        save_best_only=True,  # 최고 성능 모델만 저장
        save_weights_only=True,  # 가중치만 저장 (전체 모델 저장 X)
    )
    
    # 모델 학습 (훈련 데이터에서 10%를 검증 데이터로 사용)
    history = model.fit(
        x=x_train,  # 학습 데이터 (입력 이미지)
        y=y_train,  # 학습 데이터 (정답 레이블)
        batch_size=batch_size,  # 배치 크기 설정
        epochs=num_epochs,  # 학습 반복 횟수
        validation_split=0.1,  # 학습 데이터 중 10%를 검증 데이터로 사용
        callbacks=[checkpoint_callback],  # 체크포인트 콜백 적용
    )

In [ ]:
# 학습된 가중치 중 최적 성능 모델 불러오기
    model.load_weights(checkpoint_filepath)
    
    # 테스트 데이터에서 모델 성능 평가
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    
    # 테스트 정확도 출력
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
    return history  # 학습 기록 반환

# Vision Transformer(ViT) 모델 생성
vit_classifier = create_vit_classifier()

# ViT 모델 학습 및 평가 실행
history = run_experiment(vit_classifier)


In [ ]:
# 학습 결과 시각화 함수 정의
def plot_history(item):
    plt.plot(history.history[item], label=item)  # 학습 데이터의 손실 또는 정확도 그래프
    plt.plot(history.history["val_" + item], label="val_" + item)  # 검증 데이터의 손실 또는 정확도 그래프
    plt.xlabel("Epochs")  # x축: 에포크(Epochs)
    plt.ylabel(item)  # y축: 손실(Loss) 또는 정확도(Accuracy)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)  # 그래프 제목 설정
    plt.legend()  # 범례 추가
    plt.grid()  # 격자 표시
    plt.show()  # 그래프 출력
    
# 학습 손실 그래프 출력
plot_history("loss")

# Top-5 정확도 그래프 출력
plot_history("top-5-accuracy")